In [8]:
from nilearn.image import load_img, get_data
import numpy as np

regions = ["V5_L","V5_R","PT_L","PT_R"]
subs = [0]*23
sizes = [dict((region,0) for region in regions) for sub in subs]
values = dict()

for i in range(1,24):
    for region in regions :
        id = str(i)
        if(i<10):
            id = "0"+id
        image = get_data("masks/our_masks_2_radius=5/"+region+"_sub_"+id+".nii")
        #image = get_data("../Data/CrossMot_raw/code/visualLocalizer/"+region+"_sub_"+id+".nii")
        somme = np.sum(image)
        sizes[i-1][region] = somme
        if somme not in values :
            values[somme] = 1
        else :
            values[somme] += 1

        del image

In [4]:
from sklearn.model_selection import GridSearchCV
from load_data import *
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import pandas as pd
import numpy as np

classes = ['Up', 'Down', 'Right', 'Left']
nb_runs = 12
labels = np.array(classes*nb_runs)

maps_masked, mask_exist = load_full_data([1], 4, 12, maps_folder="brain_maps/our_maps_3", masks_folder="masks/our_masks_3_radius=10",is_from_mohamed=False, use_t_maps=True)

df = maps_masked[0]['vis'][0]['PT_R']
test_index = range(4)
train_index = range(4,48)
X_train, X_test = df[train_index], df[test_index]
y_train, y_test = labels[train_index], labels[test_index]

params={'C':[0.01,0.1,1,10,100],
'kernel':["linear"]}
params={}
gs = GridSearchCV(KNeighborsClassifier(weights="distance"), params, cv = 11)
gs.fit(X_train, y_train)
val_results = gs.cv_results_
val_params = [str(elem).replace(" ","").replace(":","=").replace("\'","").replace("{","").replace("}","") for elem in val_results['params']]
l = [str(i) for i in range(11)]
keys = ['split'+i+'_test_score' for i in l]
tab = [val_results[key] for key in keys]
means = np.mean(tab, axis=0)
val_scores = dict(zip(val_params, means.tolist()))

In [ ]:
from load_data import *
from utility import *
import sklearn
from plots import *

from sklearn.ensemble import AdaBoostClassifier
type_maps = "_t_maps_"
subjects_ids=range(1,6)
classifiers = {
    'svm':sklearn.svm.SVC(random_state=0),
    #'Ada':AdaBoostClassifier(random_state=SEED),
}
out_directory = "out/"+"our"+type_maps
for name in classifiers:
    out_dir = out_directory+str(classifiers[name])+"_"+"3_radius=10/"
    cv_group_df = retrieve_cv_metric(out_dir, "group_scores")
    cv_df = retrieve_cv_metric(out_dir, "accuracy")
    cfm_df = retrieve_cv_matrixes(out_dir)
    val_scores_df = retrieve_val_scores(out_dir)
    pvals = retrieve_pvals(out_dir, default_keys=cv_df.columns)
    plt_dir = "plots/"+"our"+type_maps+str(classifiers[name])+"_"+"3_radius=10/"
    create_directory(plt_dir)
    plotter = Plotter(plt_dir, subjects_ids)
    plotter.plot_cv_score_with_points(cv_df, pvals, chance_level = True)


In [75]:
score_per_analysis = dict()
group_by = "C" # what the x-axis of the future plot should be
group_by_values = [0.001,0.01,0.1,1,10]
str_group_by_values = [str(val) for val in group_by_values]
#TODO checker avec masks_exist
counter = dict()
compte = 0

for analysis in val_scores_df:
    score_per_analysis[analysis] = dict()
    for i, subj in enumerate(subjects_ids) :
        combs = val_scores_df[analysis][subj].replace(" ","").replace("{","").replace("}","").replace("\'","").split(",")
        for comb in combs :
            index=0
            new_key = ""
            tmp = comb.split(":")
            score=float(tmp[1])
            indiv_params = tmp[0].split("-")
            for param in indiv_params :
                key = param.split("__")[1]
                tmp_ = param.split("=")
                name = tmp_[0].split("__")[1]
                value = tmp_[1]
                if not name == group_by :
                    new_key += " - " + key
                else :
                    index = str_group_by_values.index(value)
                #print(name);print(value)
            new_key = new_key.split(" - ",1)[1]
            #print(new_key)
            compte += 1

            if new_key in score_per_analysis[analysis] :
                score_per_analysis[analysis][new_key][i,index] = score
            else :
                score_per_analysis[analysis][new_key] = np.zeros((len(subjects_ids),len(group_by_values)))
                score_per_analysis[analysis][new_key][i,0] = score



for analysis in score_per_analysis :
    for key in score_per_analysis[analysis] :
        score_per_analysis[analysis][key] = np.mean(score_per_analysis[analysis][key], axis=0)


In [2]:
from load_data import *
import sklearn

type_maps = "_t_maps_"
subjects_ids=range(1,6)
classifiers = {
    'svm':sklearn.svm.SVC(random_state=0),
    #'Ada':AdaBoostClassifier(random_state=SEED),
}
out_directory = "out/"+"our"+type_maps
for name in classifiers:
    out_dir = out_directory+str(classifiers[name])+"_"+"3_radius=10/"
    masks_exist = retrieve_masks_exist(out_dir)